# Transforma em DataFrame e limpa

Leitura do DataFrame

In [6]:
import pandas as pd

# Lê excel e transforma em DataFrame
vendas_df = pd.read_excel('../../../data/silver/2025_11_treated.xlsx')

Remove e renomeia colunas

In [7]:
# Cria tupla de sufixos de colunas necessários
sufixos_necessários = tuple(['Vendas', 'Anúncios', 'Publicidade'])

# Remove colunas desnecessárias pelo sufixo
for col in vendas_df.columns:
    if not col.endswith(sufixos_necessários):
        vendas_df = vendas_df.drop(col, axis=1)

In [8]:
# Dropa mais colunas desnecessárias
colunas_drop = ['Estado_Vendas', 'Descrição_do_status_Vendas', 'Pacote_de_diversos_produtos_Vendas', 'Pertence_a_um_kit_Vendas', 'SKU_Anúncios', '#_de_anúncio_Anúncios', 'Canal_de_venda_Anúncios', 'Receita_por_acréscimo_no_preço_(pago_pelo_comprador)_Vendas', 'Taxa_de_parcelamento_equivalente_ao_acréscimo_Vendas']
vendas_df = vendas_df.drop(colunas_drop, axis=1)

In [9]:
# Renomeia as colunas
novos_nomes = ['id_venda', 'data_venda', 'unidades_vendidas', 'receita_por_produto', 'tarifas_impostos_venda', 'receita_envio', 'tarifa_envio', 'custo_envio', 'diff_custo_envio', 'cancelamentos_reembolsos', 'total_vendas', 'mes_faturamento_tarifas', 'venda_publicidade', 'titulo_anuncio', 'variacao_anuncio', 'preco_unitario_anuncio', 'tipo_anuncio']
vendas_df.columns = novos_nomes

Filtra apenas vendas concluídas

In [10]:
# Filtra apenas colunas de vendas concluídas
vendas_df = vendas_df[
    # Valor total da venda precisa ser maior que 0
    (vendas_df['total_vendas'] != 0) &
    # É preciso que haja o título do anúncio, a ausência indica que a venda não foi concluída
    (vendas_df['titulo_anuncio'].notna()) &
    (vendas_df['titulo_anuncio'].str.strip() != '')
]

# Reseta índice do DataFrame
vendas_df = vendas_df.reset_index(drop=True)

Corrige tipos e valores

In [11]:
# Converte o id de int para string
vendas_df['id_venda'] = vendas_df['id_venda'].astype(str)

In [12]:
# Remove a hora da coluna de data
vendas_df['data_venda'] = vendas_df['data_venda'].str[:-9]

In [13]:
# Remove os espaços vazios da coluna de variação do anuncio e as chaves 'cor' e 'tamanho'
vendas_df['variacao_anuncio'] = vendas_df['variacao_anuncio'] \
                                    .str.replace(' ', '', regex=False) \
                                    .str.replace('Cor:', '', regex=False) \
                                    .str.replace('Tamanho:', '', regex=False)

In [14]:
# Cria as colunas cor kimono e tamanho kimono a partir da coluna variação anúncio
vendas_df[['cor_kimono', 'tamanho_kimono']] = vendas_df['variacao_anuncio'].str.split('|', expand=True)

# Identifica categoria de produtos pelo título do anúncio

In [15]:
# Cria lista que será copiada para as outras
lista_copia = [0] * len(vendas_df)

# Cria listas por categoria que serão adicionas ao DataFrame
str_bz = lista_copia.copy()
akai_fem = lista_copia.copy()
fx = lista_copia.copy()

In [16]:
# Itera por cada linha do DataFrame
for idx, row in vendas_df.iterrows():

    # Define título do anúncio e unidades vendidas
    titulo_anuncio = row['titulo_anuncio'].lower()
    unidades_vendidas = int(row['unidades_vendidas'])

    # Caso o anúncio seja de kimono
    if 'kimono' in titulo_anuncio:

        # Se contiver akai ou fem no nome, muda o elemento correspondente ao índice na lista akai_fem pelas unidades vendidas
        if ('akai' in titulo_anuncio) or ('feminin' in titulo_anuncio):
            akai_fem[idx] = unidades_vendidas
        
        # Se não for akai/fem, será str/bz. Então, muda o elemento correspondente ao índice na lista str_bz pelas unidades vendidas
        else:
            str_bz[idx] = unidades_vendidas

        # Caso o anuncio contenha faixa, muda o elemento correspondente ao índice na lista fx pelas unidades vendidas 
        if 'faixa' in titulo_anuncio:
            fx[idx] = unidades_vendidas

    # Se o anúncio não for de kimono, será de faixa. Então...
    # else:
    ## TODO

In [17]:
# Cria novas colunas no DataFrame de vendas
vendas_df['STR/BZ'] = str_bz
vendas_df['AKAI/FEM'] = akai_fem
vendas_df['FX'] = fx

In [18]:
# Salva DataFrame
vendas_df.to_csv('../../../data/gold/2025_11_final.csv', index=False)